# Layer Hook 验证实验

本 notebook 验证 `generate_with_dual_cache_hooked` 与原版 `generate_with_dual_cache` 的生成结果完全一致。

**验证目标：**
1. 添加 hook 后，生成结果与原版完全相同
2. Hook 能够正确收集各层的 hidden states
3. NFE (Number of Forward Evaluations) 保持一致

## 1. 环境设置

In [1]:
import os
import torch
import gc

# 设置 GPU（根据你的环境修改）
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# 切换到 llada 目录
os.chdir('llada')

# 清理 GPU 缓存
torch.cuda.empty_cache()
gc.collect()

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

CUDA available: True
GPU: NVIDIA GeForce RTX 3090 Ti
Total VRAM: 23.99 GB


## 2. 加载模型

In [2]:
from transformers import AutoTokenizer
from model.modeling_llada import LLaDAModelLM

device = 'cuda'
model_name = 'GSAI-ML/LLaDA-8B-Instruct'

print(f"Loading model: {model_name}")
model = LLaDAModelLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
).to(device).eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print("✅ Model loaded!")

# 显示模型结构信息
n_layers = model.model.config.n_layers
print(f"\nModel info:")
print(f"   Number of layers: {n_layers}")
print(f"   Hidden size: {model.model.config.d_model}")

/home/pianng/miniconda3/envs/dllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading model: GSAI-ML/LLaDA-8B-Instruct


Loading checkpoint shards: 100%|██████████| 6/6 [00:00<00:00,  7.43it/s]


✅ Model loaded!

Model info:
   Number of layers: 32
   Hidden size: 4096


## 3. 导入生成函数

In [3]:
from generate import (
    generate_with_dual_cache,
    generate_with_dual_cache_hooked,
    LayerHiddenStatesCollector
)

print("✅ Functions imported!")

✅ Functions imported!


## 4. 验证实验：对比原版与 Hook 版本

使用多个不同的 prompt 进行测试，确保结果一致。

In [4]:
def prepare_input(prompt_text, tokenizer, device):
    """准备输入 tensor"""
    m = [{"role": "user", "content": prompt_text}]
    prompt = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)
    input_ids = tokenizer(prompt)['input_ids']
    input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)
    return input_ids

def compare_generation(model, tokenizer, prompt_text, gen_params, device='cuda'):
    """
    对比原版和 hook 版本的生成结果。
    
    Returns:
        dict: 包含对比结果
    """
    input_ids = prepare_input(prompt_text, tokenizer, device)
    prompt_len = input_ids.shape[1]
    
    # 1. 原版生成
    x_orig, nfe_orig = generate_with_dual_cache(
        model, input_ids, **gen_params
    )
    answer_orig = tokenizer.decode(x_orig[0, prompt_len:], skip_special_tokens=True)
    
    # 2. Hook 版本（不启用 collector）
    x_hooked_no_collector, nfe_hooked_no = generate_with_dual_cache_hooked(
        model, input_ids, collector=None, **gen_params
    )
    answer_hooked_no = tokenizer.decode(x_hooked_no_collector[0, prompt_len:], skip_special_tokens=True)
    
    # 3. Hook 版本（启用 collector）
    collector = LayerHiddenStatesCollector()
    n_layers = collector.register_hooks(model)
    
    x_hooked, nfe_hooked, step_states = generate_with_dual_cache_hooked(
        model, input_ids, collector=collector, collect_per_step=False, **gen_params
    )
    answer_hooked = tokenizer.decode(x_hooked[0, prompt_len:], skip_special_tokens=True)
    
    # 获取最后一步的 hidden states 信息
    layer_data = collector.get_layer_hidden_states()
    collector.remove_hooks()
    
    # 对比
    tokens_match_no_collector = torch.equal(x_orig, x_hooked_no_collector)
    tokens_match_with_collector = torch.equal(x_orig, x_hooked)
    nfe_match = (nfe_orig == nfe_hooked_no == nfe_hooked)
    
    return {
        'prompt': prompt_text,
        'answer_orig': answer_orig,
        'answer_hooked_no_collector': answer_hooked_no,
        'answer_hooked_with_collector': answer_hooked,
        'nfe_orig': nfe_orig,
        'nfe_hooked_no_collector': nfe_hooked_no,
        'nfe_hooked_with_collector': nfe_hooked,
        'tokens_match_no_collector': tokens_match_no_collector,
        'tokens_match_with_collector': tokens_match_with_collector,
        'nfe_match': nfe_match,
        'n_layers_hooked': n_layers,
        'layer_inputs_collected': len(layer_data['inputs']),
        'layer_outputs_collected': len(layer_data['outputs']),
    }

In [5]:
# 测试用的 prompts
test_prompts = [
    "What is 2 + 2?",
    "Who is Newton?",
    "Lily can run 12 km/h for 4 hours. After that, she runs 6 km/h. How many km can she run in 8 hours?",
    "Explain the concept of machine learning in one sentence.",
]

# 生成参数
gen_params = {
    'steps': 32,
    'gen_length': 64,
    'block_length': 32,
    'temperature': 0.,
    'remasking': 'low_confidence',
    'threshold': 0.9,
}

print("开始验证实验...")
print("=" * 80)

all_results = []
all_passed = True

for i, prompt in enumerate(test_prompts):
    print(f"\n[Test {i+1}/{len(test_prompts)}] Prompt: {prompt[:50]}...")
    
    result = compare_generation(model, tokenizer, prompt, gen_params)
    all_results.append(result)
    
    # 检查结果
    passed = result['tokens_match_no_collector'] and result['tokens_match_with_collector'] and result['nfe_match']
    all_passed = all_passed and passed
    
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"   {status}")
    print(f"   Tokens match (no collector): {result['tokens_match_no_collector']}")
    print(f"   Tokens match (with collector): {result['tokens_match_with_collector']}")
    print(f"   NFE: orig={result['nfe_orig']}, hooked_no={result['nfe_hooked_no_collector']}, hooked={result['nfe_hooked_with_collector']}")
    print(f"   Layers hooked: {result['n_layers_hooked']}")
    print(f"   Hidden states collected: inputs={result['layer_inputs_collected']}, outputs={result['layer_outputs_collected']}")

print("\n" + "=" * 80)
if all_passed:
    print("🎉 所有测试通过！Hook 版本与原版生成结果完全一致。")
else:
    print("⚠️ 部分测试失败，请检查实现。")

开始验证实验...

[Test 1/4] Prompt: What is 2 + 2?...
   ✅ PASSED
   Tokens match (no collector): True
   Tokens match (with collector): True
   NFE: orig=5, hooked_no=5, hooked=5
   Layers hooked: 32
   Hidden states collected: inputs=32, outputs=32

[Test 2/4] Prompt: Who is Newton?...
   ✅ PASSED
   Tokens match (no collector): True
   Tokens match (with collector): True
   NFE: orig=17, hooked_no=17, hooked=17
   Layers hooked: 32
   Hidden states collected: inputs=32, outputs=32

[Test 3/4] Prompt: Lily can run 12 km/h for 4 hours. After that, she ...
   ✅ PASSED
   Tokens match (no collector): True
   Tokens match (with collector): True
   NFE: orig=32, hooked_no=32, hooked=32
   Layers hooked: 32
   Hidden states collected: inputs=32, outputs=32

[Test 4/4] Prompt: Explain the concept of machine learning in one sen...
   ✅ PASSED
   Tokens match (no collector): True
   Tokens match (with collector): True
   NFE: orig=23, hooked_no=23, hooked=23
   Layers hooked: 32
   Hidden states co

## 5. 详细对比输出

In [6]:
import pandas as pd

# 创建对比表格
comparison_data = []
for r in all_results:
    comparison_data.append({
        'Prompt': r['prompt'][:40] + '...' if len(r['prompt']) > 40 else r['prompt'],
        'NFE (orig)': r['nfe_orig'],
        'NFE (hooked)': r['nfe_hooked_with_collector'],
        'Tokens Match': '✅' if r['tokens_match_with_collector'] else '❌',
        'Layers Hooked': r['n_layers_hooked'],
    })

df = pd.DataFrame(comparison_data)
display(df)

,Prompt,NFE (orig),NFE (hooked),Tokens Match,Layers Hooked
0,What is 2 + 2?,5,5,✅,32
1,Who is Newton?,17,17,✅,32
2,Lily can run 12 km/h for 4 hours. After ...,32,32,✅,32
3,Explain the concept of machine learning ...,23,23,✅,32


## 6. 验证 Hidden States 收集

确认 collector 正确收集了各层的 hidden states。

In [7]:
# 使用一个简单的 prompt 进行详细检查
test_prompt = "What is 2 + 2?"
input_ids = prepare_input(test_prompt, tokenizer, device)

# 创建 collector 并注册 hooks
collector = LayerHiddenStatesCollector()
n_layers = collector.register_hooks(model)

print(f"Registered hooks on {n_layers} layers")

# 运行生成
x, nfe, _ = generate_with_dual_cache_hooked(
    model, input_ids, collector=collector, collect_per_step=False,
    steps=16, gen_length=32, block_length=32, temperature=0., threshold=0.9
)

# 获取 hidden states
layer_data = collector.get_layer_hidden_states()

print(f"\nCollected hidden states:")
print(f"   Number of layer inputs: {len(layer_data['inputs'])}")
print(f"   Number of layer outputs: {len(layer_data['outputs'])}")

# 检查形状
if layer_data['outputs']:
    sample_layer = list(layer_data['outputs'].keys())[0]
    sample_tensor = layer_data['outputs'][sample_layer]
    print(f"\n   Sample tensor shape (layer {sample_layer}): {sample_tensor.shape}")
    print(f"   dtype: {sample_tensor.dtype}")

collector.remove_hooks()

Registered hooks on 32 layers

Collected hidden states:
   Number of layer inputs: 32
   Number of layer outputs: 32

   Sample tensor shape (layer 0): torch.Size([1, 32, 4096])
   dtype: torch.bfloat16


In [8]:
# 显示各层 hidden states 的基本统计信息
print("各层 Hidden States 统计信息:")
print("-" * 60)

for layer_idx in sorted(layer_data['outputs'].keys()):
    tensor = layer_data['outputs'][layer_idx]
    tensor_float = tensor.float()  # 转换为 float 以便计算统计值
    
    mean_val = tensor_float.mean().item()
    std_val = tensor_float.std().item()
    min_val = tensor_float.min().item()
    max_val = tensor_float.max().item()
    
    print(f"Layer {layer_idx:2d}: mean={mean_val:+.4f}, std={std_val:.4f}, min={min_val:+.4f}, max={max_val:+.4f}")

各层 Hidden States 统计信息:
------------------------------------------------------------
Layer  0: mean=-0.0118, std=0.5781, min=-34.5000, max=+2.1250
Layer  1: mean=-0.0152, std=0.6690, min=-40.2500, max=+2.1719
Layer  2: mean=-0.0165, std=0.7223, min=-42.7500, max=+2.0469
Layer  3: mean=-0.0178, std=0.8159, min=-47.2500, max=+2.3906
Layer  4: mean=-0.0161, std=0.8788, min=-51.2500, max=+3.0000
Layer  5: mean=-0.0201, std=0.9626, min=-55.7500, max=+3.0938
Layer  6: mean=-0.0204, std=1.0173, min=-61.0000, max=+3.8906
Layer  7: mean=-0.0205, std=1.0437, min=-63.5000, max=+3.9844
Layer  8: mean=-0.0203, std=1.0534, min=-62.2500, max=+4.2500
Layer  9: mean=-0.0212, std=1.1357, min=-68.0000, max=+5.0938
Layer 10: mean=-0.0221, std=1.1500, min=-71.5000, max=+5.5938
Layer 11: mean=-0.0209, std=1.1531, min=-74.0000, max=+6.7188
Layer 12: mean=-0.0235, std=1.2016, min=-82.0000, max=+7.6875
Layer 13: mean=-0.0228, std=1.2805, min=-86.0000, max=+8.5000
Layer 14: mean=-0.0244, std=1.3256, min=-97.5000

## 7. 结论

本 notebook 验证了：

1. **生成结果一致性**: `generate_with_dual_cache_hooked` 在启用和不启用 collector 时，与原版 `generate_with_dual_cache` 生成的 token 序列完全相同

2. **NFE 一致性**: 前向传播次数保持一致

3. **Hidden States 收集**: Collector 能够正确收集所有层的输入和输出 hidden states

这为后续的 layer-level similarity 分析提供了可靠的基础。